# Example: RDF workflow

This notebook demonstrates the usage of the [rdf.py](../src/pyBiodatafuse/graph/rdf.py) module.

In [ ]:
import os
#os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))
import pandas as pd
from pyBiodatafuse.graph import rdf


### Load the sample property table

In [2]:
data = pd.read_pickle("../combined_df_all.pkl")
metadata = pd.read_pickle("../combined_metadata.pkl")
data.head(3)

,identifier,identifier.source,target,target.source,Bgee_gene_expression_levels,DISGENET_diseases,literature_based_info,MINERVA,WikiPathways,OpenTargets_reactome,OpenTargets_go,OpenTargets_gene_compounds,MolMeDB_transporter_inhibitor,PubChem_assays,StringDB_ppi
0,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...",[{'disease_name': 'Multiple pterygium syndrome...,"[{'disease_name': nan, 'id': nan, 'source': nan}]","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...","[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'compound_name': nan, 'inchikey': nan, 'smil...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...",[{'disease_name': 'Multiple pterygium syndrome...,"[{'disease_name': nan, 'id': nan, 'source': nan}]","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...","[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'compound_name': nan, 'inchikey': nan, 'smil...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
2,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...",[{'disease_name': 'Multiple pterygium syndrome...,"[{'disease_name': nan, 'id': nan, 'source': nan}]","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...","[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'compound_name': nan, 'inchikey': nan, 'smil...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."


In [3]:
print(len(data))
data.describe()

1622


,identifier,identifier.source,target,target.source,Bgee_gene_expression_levels,DISGENET_diseases,literature_based_info,MINERVA,WikiPathways,OpenTargets_reactome,OpenTargets_go,OpenTargets_gene_compounds,MolMeDB_transporter_inhibitor,PubChem_assays,StringDB_ppi
count,1622,1622,1622,1622,1622,1622,1622,1622,1622,1622,1622,1622,1622,1622,1622
unique,6,1,6,1,6,6,5,6,6,6,6,6,6,50,6
top,DMD,HGNC,ENSG00000198947,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'disease_name': 'Muscular Dystrophy, Duchenn...","{'disease_name': 'Post-COVID-19', 'id': 'C0000...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP2858', 'pathway_label': 'Ec...",[{'pathway_label': 'Striated Muscle Contractio...,"[{'go_id': 'GO:0016010', 'go_name': 'dystrophi...","[{'chembl_id': 'CHEMBL2108278', 'drugbank_id':...","[{'compound_name': nan, 'inchikey': nan, 'smil...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
freq,1600,1622,1600,1622,1600,1600,1601,1600,1600,1600,1600,1600,1600,40,1600


### Generating RDF from table
Rows with no data will be skipped:

In [4]:
nas = data[data["identifier"].isna()]
print(f"Skipped row count: {len(nas)}")
if len(nas) > 0:
    nas.sample(3)

Skipped row count: 0


The function to generate an RDF `rdflib` graph (`generate_rdf()`) takes two arguments:


In [5]:
g = rdf.generate_rdf(df = data[1500:], 
                     base_uri = "https://biodatafuse.org/example/", 
                     version_iri = "https://biodatafuse.org/example/test.owl", 
                     orcid = "https://orcid.org/0000-0002-4166-7093", 
                     author="Javier Millan Acosta", 
                     metadata = metadata)

### Print out result

In [6]:
g.serialize(format="turtle", destination='test.ttl')

<Graph identifier=N50664fd1de7543f6a0527b84a88cc38d (<class 'rdflib.graph.Graph'>)>